In [5]:
%matplotlib inline

# import libraries
import numpy as np
import scipy.optimize as optimize
import scipy.stats as stats
import matplotlib.pylab as plt
import pandas as pd
import glob

In [ ]:
def binding_curve(peptide,KD,delta_max):
    
    return delta_max*(1/(1 + KD/peptide))
    
def residuals(fit_param,obs_df):
    #fit param: array of fit aprameters.
    
    # last entry is KD
    KD = fit_param[-1]
    
    # First entries are 1-N delta_max
    delta_max = fit_param[:-1]
    
    resid_counter = 0
    residual = []
    for resid in unique(obs_df.residue_number):
        resid_df = obs_df[obs_df.residue_number == resid]
        
        resid_calc = binding_curve(resid_df.peptide,KD,delta_max[resid_counter])
        
        residual.append(resid_calc - resid_df.chemical_shift)
        
        resid_counter += 1
        
    return np.concatenate(residual)

# fit guesses
DELTA_MAX_GUESS = 1
KD_GUESS = 10
    
# read data frame
df = pd.read_csv("some_csv")

# Create array containing guesses
num_residues = len(np.unique(df.residue_number))
guesses = DELTA_MAX_GUESS*np.ones(num_residues + 1)
guesses[-1] = KD_GUESS

# Do fit
fit_param, covariance, info_dict, message, error = optimize.leastsq(residuals,
                                                                    x0=guesses,
                                                                    args=(df,),
                                                                    full_output=True)

print(fit_param)